In [1]:
from jupyter_dash import JupyterDash
from dash import Dash, html,dcc,Input, Output
import dash_leaflet as dl
import json
import httpx
from rio_tiler.io import COGReader

In [2]:
titiler_endpoint = "https://mfkh268xng.execute-api.us-east-1.amazonaws.com/"  # personal endpoint. Please be kind.
#url = "https://rapidepe123.s3.eu-west-2.amazonaws.com/GBR_Wind_Speed_100m_ZSTD.tif"
url = "https://rapidepe123.s3.eu-west-2.amazonaws.com/GBR_wind-speed_100m_COGv2.tif"


In [3]:
with COGReader(url) as cog:
    info = cog.info()
print(info.dict(exclude_none=True))

{'bounds': BoundingBox(left=-14.888427317807158, bottom=47.968268747735536, right=3.399072682193749, top=63.888268747736326), 'minzoom': 4, 'maxzoom': 9, 'band_metadata': [('1', {})], 'band_descriptions': [('1', '')], 'dtype': 'float32', 'nodata_type': 'Nodata', 'colorinterp': ['gray'], 'height': 6368, 'overviews': [2, 4, 8, 16, 32, 64], 'nodata_value': nan, 'width': 7315, 'driver': 'GTiff', 'count': 1}


In [4]:
with COGReader(url) as cog:
    stats = cog.statistics()

    # assert isinstance(meta, dict)
    # print(list(meta))
    # print(meta["1"].dict())

stat_dict = stats["1"].dict()

minv = stat_dict['min']
maxv = stat_dict['max']
print(minv)
print(maxv)

4.905625820159912
16.434234619140625


In [45]:
def get_point_value(lat,lon):

    with COGReader(url) as cog:
        values = cog.point(lon,lat)
        
    return "{:.2f}".format(float(values[0]))

x = get_point_value(55.5,-4)
print(x)

10.10


In [6]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "rescale": f"{minv},{maxv}",
        "colormap_name": "viridis"
    }
).json()


In [43]:
app = JupyterDash(__name__)

# app.layout = html.Div([
#     dl.Map([dl.TileLayer(), dl.GeoTIFFOverlay( interactive=True, url=geotiff_url, band=0, opacity=0.9)],
#            id="map", style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})])
#app.layout = html.Div([dcc.Graph(figure=fig)])
# color_domain = dict(domainMin=20, domainMax=40, colorscale=['white', 'orange', 'red'])
# color_domain2 = dict(min=20, max=40, colorscale=['white', 'orange', 'red'])
# GEOTIFF_ID = "geotiff-id"
# GEOTIFF_MARKER_ID = "geotiff-marker-id"
# def get_info(feature=None):
#     header = [html.H4("Wind Speed")]
#     if not feature:
#         return header + [html.P("Click for Wind )]
#     return header + [html.B(id="WS"), html.Br(),
#                      "{:.3f} people / mi".format(feature["properties"]["density"]), html.Sup("2")]

# Create info control.
info = html.Div( id="info", className="info",
                style={"position": "absolute", "bottom": "10px", "left": "10px", "z-index": "1000"})



IDs = ["TileMap","WindSpeed@100m"]
app.layout = html.Div([
dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[55, -4],
               zoom=5,
               id = "map",
               children=[
            dl.LayersControl([
                
                   dl.Overlay(dl.LayerGroup(dl.TileLayer(url="https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_nolabels/{z}/{x}/{y}.png",id="TileMap")),name="BaseMap",checked=True),
                   dl.Overlay(dl.LayerGroup(dl.TileLayer(url=r["tiles"][0], opacity=0.8,id="WindSpeed@100m")),name="WS@100m",checked=True),
                   dl.LayerGroup(id="layer"),
                                
                   dl.Colorbar(colorscale="viridis", width=20, height=150, min=minv, max=maxv,unit='m/s',position="bottomright"),
                   info,
                   
])
])
])

@app.callback(Output("info", "children"), [Input("map", "click_lat_lng")])
def map_click(click_lat_lng):

   

    lat= click_lat_lng[0]
    lon=click_lat_lng[1]
    
  
    

    return get_point_value(lat,lon)



app.run_server(debug=True,mode='inline')


In [8]:
print(lat)

NameError: name 'lat' is not defined

The below code looks at leafmap package 

In [ ]:
import leafmap.plotlymap as leafmap

In [ ]:
m = leafmap.Map()
m.add_basemap("Stamen.Terrain")
leafmap.fix_widget_error()
m.add_controls
m.add_cog_layer(url=url,colormap_name ="viridis" )
m.show()

[55.94919982336746, -2.4169921875000004]
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: str.format() argument after * must be an iterable, not NoneType

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'list' object is not callable

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'list' object is not callable



In [ ]:
import requests    # To install: pip install requests

geotiff_url_tempAWS = create_presigned_url('rapidepe123', 'tz850.tiff')
if geotiff_url_tempAWS is not None:
    response = requests.get(geotiff_url_tempAWS)

In [ ]:
#read geotiff using rioxarray and turn into xarray.Dataarray, reporject to ESPG:3857 (see Javascript code below to correct tooltip/hover)

geotiff_file = r"C:/Users/mdavi/OneDrive/Documents/GitHub/Plotly_DASH_Leaflet/ResourceFiles/WS100m.tif"
#dataarray = rxr.open_rasterio(geotiff_file).rio.reproject('EPSG:3857')
dataarray = rxr.open_rasterio(geotiff_file)

#geotiff_url = 'https://opendata.digitalglobe.com/events/california-fire-2020/pre-event/2018-02-16/pine-gulch-fire20/1030010076004E00.tif'
#dl.GeoTIFFOverlay(id=GEOTIFF_ID, interactive=True, url=geotiff_file, band=0, opacity=0.9)

In [ ]:
import base64
def create_onedrive_directdownload (onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl

